## 建立資料庫引擎

In [1]:
from sqlalchemy import create_engine

engine = create_engine('sqlite+pysqlite:///data/nobel_prize", echo=True, future=True')

## 定義資料庫表格

這裡可參考：https://docs.sqlalchemy.org/en/13/orm/extensions/declarative/basic_use.html

In [2]:
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()

In [19]:
from sqlalchemy import Column, Integer, String, Enum

class Winner(Base):

    # 命名SQL表格
    __tablename__ = 'winners'

    id = Column(Integer, primary_key=True)
    name = Column(String)
    category = Column(String)
    year = Column(Integer)
    nationality = Column(String)
    sex = Column(Enum('male', 'female'))

    def __repr__(self):
        return "<Winner(name='%s', category='%s', year='%s')>"\
            %(self.name, self.category, self.year)

/tmp/ipykernel_8429/2377975308.py:3: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.Winner, and will be replaced in the string-lookup table.
  class Winner(Base):


InvalidRequestError: Table 'winners' is already defined for this MetaData instance.  Specify 'extend_existing=True' to redefine options and columns on an existing Table object.

In [4]:

Base.metadata.create_all(engine)

## 加入資料

In [5]:
nobel_winners = [
    {'category':'Physics',
    'name':'Albert Einstein',
    'nationality':'Swiss',
    'sex':'male',
    'year':1921,},

    {'category':'Physics',
    'name':'Paul Dirac',
    'nationality':'British',
    'sex':'male',
    'year':1933,},

    {'category':'Physics',
    'name':'Marie Curie',
    'nationality':'Polish',
    'sex':'female',
    'year':1911,},
]

將物件加入工作階段

In [6]:
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=engine)
session = Session()

albert = Winner(**nobel_winners[0])
session.add(albert)

#new代表剛剛加入此工作階段的任何項目集合
session.new   

IdentitySet([<Winner(name='Albert Einstein', category='Physics', year='1921')>])

將物件從工作階段移除

In [12]:
session.expunge(albert)
session.new

#如果使用expunge_all()則是移除工作階段所有新加入的物件

InvalidRequestError: Instance <Winner at 0x7f5fec6e7b80> is not present in this Session

將資料加入工作階段並提交到資料庫

In [8]:
winner_rows = [Winner(**w) for w in nobel_winners]
session.add_all(winner_rows)

#commit之後才會真正改變資料庫內容
# session.commit()

## 由資料庫查詢資料

SQLAlchemy官方文件：https://docs.sqlalchemy.org/en/14/orm/query.html

In [9]:
# 計算表格有幾列
n = session.query(Winner).count()
print(n)

# 諾貝爾獎瑞士人得主有誰
result = session.query(Winner).filter_by(nationality='Swiss')
print(list(result))

# 不是瑞士人的物理獎得主有誰
result2 = session.query(Winner).filter(Winner.category == 'Physics', Winner.nationality != 'Swiss')
print(list(result2))

#以年份排序
result3 = session.query(Winner).order_by('year')
print(list(result3))

9
[<Winner(name='Albert Einstein', category='Physics', year='1921')>, <Winner(name='Albert Einstein', category='Physics', year='1921')>, <Winner(name='Albert Einstein', category='Physics', year='1921')>]
[<Winner(name='Paul Dirac', category='Physics', year='1933')>, <Winner(name='Marie Curie', category='Physics', year='1911')>, <Winner(name='Paul Dirac', category='Physics', year='1933')>, <Winner(name='Marie Curie', category='Physics', year='1911')>, <Winner(name='Paul Dirac', category='Physics', year='1933')>, <Winner(name='Marie Curie', category='Physics', year='1911')>]
[<Winner(name='Marie Curie', category='Physics', year='1911')>, <Winner(name='Marie Curie', category='Physics', year='1911')>, <Winner(name='Marie Curie', category='Physics', year='1911')>, <Winner(name='Albert Einstein', category='Physics', year='1921')>, <Winner(name='Albert Einstein', category='Physics', year='1921')>, <Winner(name='Albert Einstein', category='Physics', year='1921')>, <Winner(name='Paul Dirac', ca

丟棄整個表格

In [ ]:
Winner.__table__.drop(engine)

將SQLAlchemy實體轉成dict

In [10]:
def inst_to_dict(inst, delete_id=True):
    dat = {}
    for column in inst.__table__.columns:
        dat[column.name] = getattr(inst, column.name)
    if delete_id:
        dat.pop('id')
    
    return dat

In [11]:
winners_rows = session.query(Winner)
nobel_winners = [inst_to_dict(w) for w in winner_rows]

nobel_winners

[{'name': 'Albert Einstein',
  'category': 'Physics',
  'year': 1921,
  'nationality': 'Swiss',
  'sex': 'male'},
 {'name': 'Paul Dirac',
  'category': 'Physics',
  'year': 1933,
  'nationality': 'British',
  'sex': 'male'},
 {'name': 'Marie Curie',
  'category': 'Physics',
  'year': 1911,
  'nationality': 'Polish',
  'sex': 'female'}]

### 模組：Dataset - 簡化SQL存取程序

官方文件：https://dataset.readthedocs.io/en/latest/install.html

In [25]:
import dataset

db = dataset.connect('sqlite:///data/nobel_prize.db')

In [26]:
wtable = db['winners']
winners = wtable.find()
winners = list(winners)
print(winners)

[OrderedDict([('id', 1), ('name', 'Albert Einstein'), ('category', 'Physics'), ('year', 1921), ('nationality', 'Swiss'), ('sex', 'male')]), OrderedDict([('id', 2), ('name', 'Paul Dirac'), ('category', 'Physics'), ('year', 1933), ('nationality', 'British'), ('sex', 'male')]), OrderedDict([('id', 3), ('name', 'Marie Curie'), ('category', 'Physics'), ('year', 1911), ('nationality', 'Polish'), ('sex', 'female')]), OrderedDict([('id', 4), ('name', 'Albert Einstein'), ('category', 'Physics'), ('year', 1921), ('nationality', 'Swiss'), ('sex', 'male')]), OrderedDict([('id', 5), ('name', 'Paul Dirac'), ('category', 'Physics'), ('year', 1933), ('nationality', 'British'), ('sex', 'male')]), OrderedDict([('id', 6), ('name', 'Marie Curie'), ('category', 'Physics'), ('year', 1911), ('nationality', 'Polish'), ('sex', 'female')])]


In [30]:
wtable = db['winners']
wtable.drop()

list(wtable.find())

[]

資料庫的transaction

參考文件：https://dataset.readthedocs.io/en/latest/quickstart.html?highlight=tx#using-transactions

In [31]:
with db as tx:
    for w in nobel_winners:
        tx['winners'].insert(w)

list(db['winners'].find())

#順序可被OrderedDict記住

/home/megoo/anaconda3/envs/py39/lib/python3.9/site-packages/dataset/table.py:313: RuntimeWarning: Changing the database schema inside a transaction in a multi-threaded environment is likely to lead to race conditions and synchronization issues.
  warnings.warn(


[OrderedDict([('id', 1),
              ('name', 'Albert Einstein'),
              ('category', 'Physics'),
              ('year', 1921),
              ('nationality', 'Swiss'),
              ('sex', 'male')]),
 OrderedDict([('id', 2),
              ('name', 'Paul Dirac'),
              ('category', 'Physics'),
              ('year', 1933),
              ('nationality', 'British'),
              ('sex', 'male')]),
 OrderedDict([('id', 3),
              ('name', 'Marie Curie'),
              ('category', 'Physics'),
              ('year', 1911),
              ('nationality', 'Polish'),
              ('sex', 'female')])]

#### freeze

現在已獨立為一個套件：datafreeze

要另外安裝：pip install datafreeze

參考：https://github.com/pudo/datafreeze

freeze可把某次SQL的查詢結果轉成JSON或CSV的檔案

In [34]:
from datafreeze import freeze

winners = db['winners'].find()

freeze(winners, format='csv', filename='data/nobel_winners_ds.csv')
open('data/nobel_winners_ds.csv').read()

'id,name,category,year,nationality,sex\n1,Albert Einstein,Physics,1921,Swiss,male\n2,Paul Dirac,Physics,1933,British,male\n3,Marie Curie,Physics,1911,Polish,female\n'